In [6]:
!pip install keras

     |████████████████████████████████| 276 kB 4.5 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0


In [7]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.layers import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


ModuleNotFoundError: ignored

In [5]:
!unzip boh.zip 

Archive:  boh.zip
replace boh/Bald Eagle (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: boh/Bald Eagle (1).jpg  
  inflating: boh/Bald Eagle (2).jpg  
  inflating: boh/Bald Eagle (3).jpg  
  inflating: boh/Bald Eagle (4).jpg  
  inflating: boh/Bald Eagle (5).jpg  
  inflating: boh/Bald Eagle (6).jpg  
  inflating: boh/Bald Eagle (7).jpg  
  inflating: boh/Bald Eagle (8).jpg  
  inflating: boh/Bald Eagle (9).jpg  
  inflating: boh/Church (71).JPEG    
  inflating: boh/Church (72).JPEG    
 extracting: boh/Church (73).JPEG    
  inflating: boh/Church (74).JPEG    
 extracting: boh/Church (75).JPEG    
 extracting: boh/Church (76).JPEG    
  inflating: boh/Church (77).JPEG    
  inflating: boh/Church (78).JPEG    
  inflating: boh/Church (79).JPEG    
  inflating: boh/Cuban Tody (10).jpg  
  inflating: boh/Cuban Tody (19).jpg  
  inflating: boh/Cuban Tody (26).jpg  
  inflating: boh/Cuban Tody (27).jpg  
  inflating: boh/Cuban Tody (28).jpg  
  inflating: boh/Cuban Tody (8)

In [ ]:
!unzip finetuning_test_.zip # rename as test

In [ ]:
!unzip finetuning_train.zip # rename as train

In [21]:
# Get training images (resized to 256)
X = []
path = '/content/boh/'
for filename in os.listdir(path):
    X.append(img_to_array(load_img(path+filename, target_size=(256,256), color_mode='grayscale'))) 
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [22]:
Xtrain.shape

(41, 256, 256, 1)

In [8]:
'''
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/content/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
'''

In [30]:
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.compat.v1.get_default_graph()

inception_json = inception.to_json()
with open("inception.json", "w") as json_file:
    json_file.write(inception_json)
inception.save_weights("inception_weights.h5")

In [ ]:
'''
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
'''

In [24]:
embed_input = Input(shape=(1000,))

# Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)


# Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 


# Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [39]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
      json_file = open('inception.json','r')
      loaded_model_json = json_file.read()
      json_file.close()
      loaded_model = tf.keras.models.model_from_json(loaded_model_json)
      #load weights into new model
      #loaded_model.load_weights("color_tensorflow_real_mode.h5")
      loaded_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
      embed = loaded_model.predict(grayscaled_rgb_resized)
    return embed

def image_a_b_gen(batch_size, Xtrain, datagen):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(batch)
        grayscaled_rgb = grayscaled_rgb.reshape(grayscaled_rgb.shape[0], 256, 256, 3)
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(grayscaled_rgb)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        # return one value at time
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [26]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 5

In [32]:
# Train model      
tensorboard = TensorBoard(log_dir="colorized")
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
#model.load_weights("color_tensorflow_real_mode1.h5")
model.fit(image_a_b_gen(batch_size, Xtrain, datagen), callbacks=[tensorboard], epochs=1000, steps_per_epoch=2,verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


TypeError: ignored

In [ ]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("color_tensorflow_real_mode1.h5")

In [12]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
json_file.close()
loaded_model.save_weights("color_tensorflow_real_mode.h5")

In [35]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        json_file = open('inception.json','r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = tf.keras.models.model_from_json(loaded_model_json)
        #load weights into new model
        #loaded_model.load_weights("color_tensorflow_real_mode.h5")
        loaded_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [ ]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
      json_file = open('inception.json','r')
      loaded_model_json = json_file.read()
      json_file.close()
      loaded_model = tf.keras.models.model_from_json(loaded_model_json)
      #load weights into new model
      #loaded_model.load_weights("color_tensorflow_real_mode.h5")
      loaded_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
      embed = loaded_model.predict(grayscaled_rgb_resized)
    return embed

In [41]:
json_file = open('model.json', 'r')
loaded_model2_json = json_file.read()
loaded_model2 = tf.keras.models.model_from_json(loaded_model2_json)
json_file.close()
loaded_model2.load_weights("color_tensorflow_real_mode.h5")


# Make predictions on validation images
color_me = []
for filename in os.listdir(path):
    color_me.append(img_to_array(load_img(path+filename, target_size=(256,256), color_mode='grayscale'))) 
print(color_me[0].shape)
color_me = np.array(color_me, dtype=float)
print(color_me.shape)
color_me = 1.0/255*color_me
grayscaled_rgb = gray2rgb(color_me)
grayscaled_rgb = grayscaled_rgb.reshape(grayscaled_rgb.shape[0], 256, 256, 3)
color_me_embed = create_inception_embedding(grayscaled_rgb)
color_me = rgb2lab(grayscaled_rgb)
color_me = color_me[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = loaded_model2.predict([color_me, color_me_embed])
output = output * 128


# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img12_"+str(i)+".png", lab2rgb(cur))

(256, 256, 1)
(41, 256, 256, 1)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ValueError: ignored